Schürmann and Sigg
===========

This notebook is used to generate the results for the evaluation of the scheme by Schürmann and Sigg (Schürmann, Dominik, and Stephan Sigg. "Secure communication based on ambient audio." IEEE Transactions on mobile computing 12, no. 2 (2013): 358-370.). In the code, this scheme is called "audioFingerprint", in the paper it is evaluated in section 4.2.

First, we have to import some libraries:

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import copy
import pandas as pd
from dateutil import parser
from datetime import datetime
import numpy as np
import gzip
import json
from glob import glob
from multiprocessing import Pool
import random
import seaborn as sns
from itertools import zip_longest, combinations
from os import makedirs
from os.path import isfile
import math
from pprint import pprint

Define a series of constants so that the system can find all files. Change the paths to whatever works on your system:

In [ ]:
# File system constants
PREFIX="/media/seemoo/data/zia-data/results/"
PREFIX_RAW="/media/seemoo/data/zia-data/raw/"

PREFIX_PLOTS='/home/seemoo/plots/img'
PREFIX_JSON='/home/seemoo/plots/json'

Additional internal constants for use in the code, leave these alone unless you know what you are doing. They are documented in more detail in the Sound-Proof notebook.

In [ ]:
# Scenarios
S_CAR = 'CarExp'
S_OFFICE = 'OfficeExp'
S_MOBILE = 'MobileExp'

# Subscenarios
SU_PARKED = 'parked'
SU_CITY = 'city'
SU_HIGHWAY = 'highway'

SU_WDAY = 'weekday'
SU_NIGHT = 'night'
SU_WEND = 'weekend'

# Sensor lists
CAR_SENSORS1 = ["01", "02", "03", "04", "05", "06"]
CAR_SENSORS2 = ["07", "08", "09", "10", "11", "12"]
OFFICE_SENSORS1 = ["01", "02", "03", "04", "05", "06", "07", "08"]
OFFICE_SENSORS2 = ["09", "10", "11", "12", "13", "14", "15", "16"]
OFFICE_SENSORS3 = ["17", "18", "19", "20", "21", "22", "23", "24"]


# Features
TIME_FREQ_DIST = 'timeFreqDistance'
NOISE_FP = 'noiseFingerprint'
SOUNDPROOF = 'soundProofXcorr'
AUDIO_FP = 'audioFingerprint'

TRUONG = 'ble_wifi_truong'
SHRESTHA = 'temp_hum_press_shrestha'


# Intervals
INTERVALS = ['5sec', '10sec', '15sec', '30sec', '1min', '2min']
# Colocation arrays
# These arrays are used to represent which sensors were considered colocated
# in the two scenarios.
#                     1  2  3  4  5  6  7  8  9 10 11 12
COLO_CAR = np.array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 1
                     [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 2
                     [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 3
                     [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 4
                     [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 5
                     [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 6
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],  # 7
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],  # 8 
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],  # 9
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],  # 10
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],  # 11
                     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]) # 12

# Also prepare colocation matrix for office, using a script to make it more concise.
COLO_OFFICE = np.zeros((24, 24))
for i in range(0, 8):
    for x in range(0, 8):
        COLO_OFFICE[i, x] = 1
for i in range(8, 16):
    for x in range(8, 16):
        COLO_OFFICE[i, x] = 1
for i in range(16, 24):
    for x in range(16, 24):
        COLO_OFFICE[i, x] = 1

# Colocation information for mobile sensor scenario
COLO_MOBILE = {}
COLO_MOBILE[2] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]
COLO_MOBILE[3] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]
COLO_MOBILE[4] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[5] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 13, 51, 0), 1),
                  (datetime(2018, 10, 21, 13, 51, 0), datetime(2018, 10, 21, 13, 55, 0), 2),
                  (datetime(2018, 10, 21, 13, 55, 0), datetime(2018, 10, 21, 14, 2, 0), 1),
                  (datetime(2018, 10, 21, 14, 2, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                  (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 2),
                  (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 7, 0), 1),
                  (datetime(2018, 10, 21, 16, 7, 0), datetime(2018, 10, 21, 16, 9, 0), 2),
                  (datetime(2018, 10, 21, 16, 9, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[6] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 13, 51, 0), 1),
                  (datetime(2018, 10, 21, 13, 51, 0), datetime(2018, 10, 21, 13, 55, 0), 2),
                  (datetime(2018, 10, 21, 13, 55, 0), datetime(2018, 10, 21, 14, 2, 0), 1),
                  (datetime(2018, 10, 21, 14, 2, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                  (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 2),
                  (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 7, 0), 1),
                  (datetime(2018, 10, 21, 16, 7, 0), datetime(2018, 10, 21, 16, 9, 0), 2),
                  (datetime(2018, 10, 21, 16, 9, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[7] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 14, 2, 0), 1),
                  (datetime(2018, 10, 21, 14, 2, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                  (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 2),
                  (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[8] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 14, 12, 0), 1),
                  (datetime(2018, 10, 21, 14, 12, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                  (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 2),
                  (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 16, 0), 1),
                  (datetime(2018, 10, 21, 16, 16, 0), datetime(2018, 10, 21, 16, 25, 0), 2),
                  (datetime(2018, 10, 21, 16, 25, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[9] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 14, 12, 0), 1),
                  (datetime(2018, 10, 21, 14, 12, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                  (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 2),
                  (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 16, 0), 1),
                  (datetime(2018, 10, 21, 16, 16, 0), datetime(2018, 10, 21, 16, 25, 0), 2),
                  (datetime(2018, 10, 21, 16, 25, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[10] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 14, 20, 0), 1),
                   (datetime(2018, 10, 21, 14, 20, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                   (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 2),
                   (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[11] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 2)]
COLO_MOBILE[12] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 2)]
COLO_MOBILE[13] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 2)]
COLO_MOBILE[14] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 2)]

COLO_MOBILE[15] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 10, 48, 0), 2),
                   (datetime(2018, 10, 21, 10, 48, 0), datetime(2018, 10, 21, 10, 52, 0), 3),
                   (datetime(2018, 10, 21, 10, 52, 0), datetime(2018, 10, 21, 12, 9, 0), 2),
                   (datetime(2018, 10, 21, 12, 9, 0), datetime(2018, 10, 21, 12, 49, 0), 1),
                   (datetime(2018, 10, 21, 12, 49, 0), datetime(2018, 10, 21, 14, 17, 0), 2),
                   (datetime(2018, 10, 21, 14, 17, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                   (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 1),
                   (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 25, 0), 2),
                   (datetime(2018, 10, 21, 16, 25, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[16] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 9, 28, 0), 1),
                   (datetime(2018, 10, 21, 9, 28, 0), datetime(2018, 10, 21, 10, 48, 0), 2),
                   (datetime(2018, 10, 21, 10, 48, 0), datetime(2018, 10, 21, 10, 52, 0), 3),
                   (datetime(2018, 10, 21, 10, 52, 0), datetime(2018, 10, 21, 12, 9, 0), 2),
                   (datetime(2018, 10, 21, 12, 9, 0), datetime(2018, 10, 21, 12, 49, 0), 1),
                   (datetime(2018, 10, 21, 12, 49, 0), datetime(2018, 10, 21, 14, 17, 0), 2),
                   (datetime(2018, 10, 21, 14, 17, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                   (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 1),
                   (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 25, 0), 2),
                   (datetime(2018, 10, 21, 16, 25, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[17] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 14, 17, 0), 2),
                   (datetime(2018, 10, 21, 14, 17, 0), datetime(2018, 10, 21, 15, 0, 0), 3),
                   (datetime(2018, 10, 21, 15, 0, 0), datetime(2018, 10, 21, 16, 4, 0), 1),
                   (datetime(2018, 10, 21, 16, 4, 0), datetime(2018, 10, 21, 16, 35, 0), 2),
                   (datetime(2018, 10, 21, 16, 35, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]

COLO_MOBILE[18] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]
COLO_MOBILE[19] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]
COLO_MOBILE[20] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]
COLO_MOBILE[21] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]

COLO_MOBILE[22] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 9, 28, 0), 1),
                   (datetime(2018, 10, 21, 9, 28, 0), datetime(2018, 10, 21, 12, 13, 0), 3),
                   (datetime(2018, 10, 21, 12, 13, 0), datetime(2018, 10, 21, 12, 46, 0), 1),
                   (datetime(2018, 10, 21, 12, 46, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]

COLO_MOBILE[23] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 9, 28, 0), 1),
                   (datetime(2018, 10, 21, 9, 28, 0), datetime(2018, 10, 21, 12, 13, 0), 3),
                   (datetime(2018, 10, 21, 12, 13, 0), datetime(2018, 10, 21, 12, 46, 0), 1),
                   (datetime(2018, 10, 21, 12, 46, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]

COLO_MOBILE[24] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 17, 30, 0), 3)]

COLO_MOBILE[25] = [(datetime(2018, 10, 21, 8, 30, 0), datetime(2018, 10, 21, 14, 5, 0), 1),
                   (datetime(2018, 10, 21, 14, 5, 0), datetime(2018, 10, 21, 15, 1, 0), 2),
                   (datetime(2018, 10, 21, 15, 1, 0), datetime(2018, 10, 21, 16, 7, 0), 3),
                   (datetime(2018, 10, 21, 16, 7, 0), datetime(2018, 10, 21, 17, 30, 0), 1)]


# List all available results with their respective available parameters
RESULTS = {
    NOISE_FP: {
        "intervals": ['5sec', '10sec', '15sec', '30sec', '1min', '2min'],
        "modalities": ["audio"],
        "features": ["fingerprints_similarity_percent"]
    },
    SOUNDPROOF: {
        "intervals": ['5sec', '10sec', '15sec', '30sec', '1min', '2min'],
        "modalities": ["audio"],
        "features": ["max_xcorr"]
    },
    AUDIO_FP: {
        "intervals": ['5sec', '10sec', '15sec', '30sec', '1min', '2min'],
        "modalities": ["audio"],
        "features": ["fingerprints_similarity_percent"]
    },
    TIME_FREQ_DIST: {
        "intervals": ['5sec', '10sec', '15sec', '30sec', '1min', '2min'],
        "modalities": ["audio"],
        "features": ["max_xcorr", "time_freq_dist"]
    },
    SHRESTHA: {
        "intervals": ["."],
        "modalities": ["temp", "hum", "press"],
        "features": ["hamming_dist"]
    },
    TRUONG: {
        "intervals": ['10sec', '30sec'],
        "modalities": ["ble", "wifi"],
        "features": ["euclidean", "jaccard", "mean_exp", "mean_hamming", "sum_squared_ranks"]
    }
}

Lets first set up a few utility functions.

Functions for loading files, and their helper functions:

In [ ]:
# File loading
def load_file(path):
    """Load results from a specific file and return them as python dict."""
    if path.endswith('.gz'):
        with gzip.open(path, 'rt') as fo:
            j = json.loads(fo.read())
            return j['results']
    else:
        with open(path, 'rt') as fo:
            j = json.loads(fo.read())
            return j['results']


# Generate a path to a result file
def generate_path(scenario, modality, feature, interval, sensor1, sensor2, day=None):
    def pad_sensor(sensor):
        return "Sensor-" + str(sensor).zfill(2)
    if day is None:
        return '/'.join([PREFIX, scenario, pad_sensor(sensor1), modality, feature, interval, pad_sensor(sensor2) + ".json.gz"])
    else:
        return '/'.join([PREFIX, scenario, day, pad_sensor(sensor1), modality, feature, interval, pad_sensor(sensor2) + ".json.gz"])


def generate_summary_path(scenario, modality, feature, interval, sensor1, day=None, subscenario=None):
    def pad_sensor(sensor):
        return "Sensor-" + str(sensor).zfill(2)
    if subscenario is not None:
        filename = "Summary-{}.json.gz".format(subscenario)
    else:
        filename = "Summary.json.gz"
    if day is None:
        return '/'.join([PREFIX, scenario, pad_sensor(sensor1), modality, feature, interval, filename])
    else:
        return '/'.join([PREFIX, scenario, day, pad_sensor(sensor1), modality, feature, interval, filename])

Managing the loaded data

In [ ]:
# Separate out subscenarios
def is_subscenario(scenario, subscenario, dt_tstamp):
    """Helper function to determine if a specific provided time is in a specific subscenario.
    
    :param scenario: The scenario (office or car)
    :param subscenario: The subscenario we are interested in
    :param dt_tstamp: The datetime object that should be checked.
    :return: True if dt is within the subscenario timeframe, else False."""
    if subscenario is None:
        return True
    if scenario == S_CAR:
        INCLUDE_INTERVALS = [(datetime(2017, 11, 23, 14, 40, 0), datetime(2017, 11, 23, 14, 46, 0), SU_PARKED),
                             (datetime(2017, 11, 23, 14, 46, 0), datetime(2017, 11, 23, 15, 15, 0), SU_CITY),
                             (datetime(2017, 11, 23, 15, 15, 0), datetime(2017, 11, 23, 15, 18, 0), SU_PARKED),
                             (datetime(2017, 11, 23, 15, 18, 0), datetime(2017, 11, 23, 15, 55, 0), SU_HIGHWAY),
                             (datetime(2017, 11, 23, 15, 55, 0), datetime(2017, 11, 23, 16, 25, 0), SU_CITY),
                             (datetime(2017, 11, 23, 16, 25, 0), datetime(2017, 11, 23, 16, 43, 0), SU_HIGHWAY),
                             (datetime(2017, 11, 23, 16, 43, 0), datetime(2017, 11, 23, 17, 5, 0), SU_PARKED),
                             (datetime(2017, 11, 23, 17, 5, 0), datetime(2017, 11, 23, 17, 18, 0), SU_HIGHWAY),
                             (datetime(2017, 11, 23, 17, 18, 0), datetime(2017, 11, 23, 17, 31, 0), SU_CITY),
                             (datetime(2017, 11, 23, 17, 31, 0), datetime(2017, 11, 23, 17, 50, 0), SU_PARKED)]
    elif scenario == S_OFFICE:
        INCLUDE_INTERVALS = [(datetime(2017, 11, 27, 8, 0, 0), datetime(2017, 11, 27, 21, 0, 0), SU_WDAY),
                             (datetime(2017, 11, 27, 21, 0, 0), datetime(2017, 11, 28, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 11, 28, 8, 0, 0), datetime(2017, 11, 28, 21, 0, 0), SU_WDAY),
                             (datetime(2017, 11, 28, 21, 0, 0), datetime(2017, 11, 29, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 11, 29, 8, 0, 0), datetime(2017, 11, 29, 21, 0, 0), SU_WDAY),
                             (datetime(2017, 11, 29, 21, 0, 0), datetime(2017, 11, 30, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 11, 30, 8, 0, 0), datetime(2017, 11, 30, 21, 0, 0), SU_WDAY),
                             (datetime(2017, 11, 30, 21, 0, 0), datetime(2017, 12, 1, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 12, 1, 8, 0, 0), datetime(2017, 12, 1, 21, 0, 0), SU_WDAY),
                             (datetime(2017, 12, 1, 21, 0, 0), datetime(2017, 12, 2, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 12, 2, 8, 0, 0), datetime(2017, 12, 2, 21, 0, 0), SU_WEND),
                             (datetime(2017, 12, 2, 21, 0, 0), datetime(2017, 12, 3, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 12, 3, 8, 0, 0), datetime(2017, 12, 3, 21, 0, 0), SU_WEND),
                             (datetime(2017, 12, 3, 21, 0, 0), datetime(2017, 12, 4, 8, 0, 0), SU_NIGHT),
                             (datetime(2017, 12, 4, 8, 0, 0), datetime(2017, 12, 4, 21, 0, 0), SU_WDAY),
                             (datetime(2017, 12, 4, 21, 0, 0), datetime(2017, 12, 5, 8, 0, 0), SU_WDAY)]
    else:
        assert False, "Unknown scenario provided: " + str(scenario)
    for ts_start, ts_end, ts_subscen in INCLUDE_INTERVALS:
        if ts_start < dt_tstamp <= ts_end:
            return ts_subscen == subscenario
    assert False, "No matching timeframe found. dt=" + str(dt)


def is_colocated(sensor1, sensor2, scenario, dt=None):
    """Helper function to determine if two sensors are colocated in a specific scenario and time.
    
    :param sensor1: The first sensor number, as int
    :param sensor2: The second sensor number, as int
    :param scenario: The Scenario to consider
    :param dt: A timestamp as string for the time to consider. Only required for mobile scenario
    :return: true if devices are colocated, otherwise false."""
    # Car and office are simple lookups in their colocation tables
    if scenario == S_CAR:
        return COLO_CAR[sensor1-1][sensor2-1] == 1
    if scenario == S_OFFICE:
        return COLO_OFFICE[sensor1-1][sensor2-1] == 1
    
    # Mobile scenario is more involved due to mobile sensors
    if scenario == S_MOBILE:
        # Ensure a dt was provided
        assert dt is not None
        # Prepare state for locations of the three sensors
        s1_loc = -1
        s2_loc = -1
        
        # Lookup location of sensor 1 at the provided time
        for ts_start, ts_end, ts_loc in COLO_MOBILE[sensor1]:
            if ts_start <= dt < ts_end:
                s1_loc = ts_loc
                break
        
        # dito for sensor 2
        for ts_start, ts_end, ts_loc in COLO_MOBILE[sensor2]:
            if ts_start <= dt < ts_end:
                s2_loc = ts_loc
                break
        
        # Ensure we have sane locations for both
        assert s1_loc != -1
        assert s2_loc != -1
        # If both are in same location => colocated. Return
        return s1_loc == s2_loc
    # Something is wrong, an unknown scenario identifier was provided
    assert False, "Unknown scenario provided: " + scenario

Loading the actual data from the disk and putting it into the right format:

In [ ]:
# Load all for feature and params
def load_audio_feature(feature, interval, scenario, subscenario=None, strip=None):
    """Load an audio feature from the disk and return it as a dictionary.
    
    :param feature: The feature (e.g., SOUNDPROOF, AUDIO_FP, ...) to load
    :param interval: The interval (10 seconds, 30 seconds, ...) to load
    :param scenario: The scenario (Car or Office) to load
    :param subscenario: The subscenario to load, or None to load the full dataset.
    :param strip: If set, strip all keys except this one from the results (to save memory). When providing a string,
        it will be used as a key. When providing a List of strings, all strings will be used as keys.
    :return: The data as a dict
    """
    # Ensure that the scenario is valid
    assert scenario in [S_CAR, S_OFFICE, S_MOBILE]
    # Logic for car scenario
    if scenario != S_OFFICE:
        rv = {}
        # Set upper and lower bounds for sensor numbers
        if scenario == S_CAR:
            lower_bound = 1
            upper_bound = 12
        elif scenario == S_MOBILE:
            lower_bound = 2
            upper_bound = 25
        # Go through all sensors in the experiment
        # sensors = [2, 3, 4, 11, 12, 13, 14, 18, 19, 20, 21]
        # for i in range(len(sensors)):
        for s1 in range(lower_bound, upper_bound + 1, 1):
            # s1 = sensors[i]
            rv[s1] = {}
            # Go through all pairs this sensor is involved in as the left-hand sensor
            for s2 in range(s1 + 1, upper_bound + 1, 1):
            # for j in range(i+1, len(sensors)):
                # s2 = sensors[j]
                # Generate the path where we expect the result file to be
                path = generate_path(scenario, "audio/", feature, interval, s1, s2)
                
                # If we are not supposed to strip values from the results, simply load the data
                if strip is None:
                    # We have not yet implemented loading subscenarios without stripping
                    assert subscenario is None, "Unsupported combination of parameters"
                    
                    # Load the results
                    results = load_file(path)
                    for k in results.keys():
                        ts = parser.parse(k)
                        rv[s1][s2][ts] = results[k]
                    # rv[s1][s2] = load_file(path)
                    
                # We should strip data from the results.
                else:
                    # Load data
                    results = load_file(path)
                    # Prepare dictionary
                    rv[s1][s2] = {}
                    
                    # For all keys, only include them if they are wanted
                    for k in results.keys():
                        ts = parser.parse(k)
                        # If the key is not in the desired subscenario, skip it entirely
                        if not is_subscenario(scenario, subscenario, ts):
                            continue
                        
                        # If we have a list of desired keys, include all these keys
                        if type(strip) is list:
                            rv[s1][s2][ts] = {s: results[k][s] for s in strip}
                        # Otherwise, we only include the "strip" key.
                        else:
                            rv[s1][s2][ts] = {strip: results[k][strip]}
        # Return the result
        return rv
    # Logic for office scenario (different folder structure requires different code)
    else:
        rv = {}
        # Once again, go through all combinations of sensors
        for s1 in range(1, 25, 1):
            rv[s1] = {}
            for s2 in range(s1 + 1, 25, 1):
                rv[s1][s2] = {}
                
                # Generate one path to load for every day of the office experiment
                for day in ["audio/1_0-24h/", "audio/2_24-48h/", "audio/3_48-72h/", "audio/4_72-96h/", "audio/5_96-120h/", "audio/6_120-144h/", "audio/7_144-168h/"]:
                    path = generate_path(scenario, "audio/", feature, interval, s1, s2, day=day)
                    
                    # If we're not stripping the results, simply load the data
                    if strip is None:
                        assert subscenario is None, "Unsupported combination of parameters"
                        rv[s1][s2][day] = load_file(path)
                    
                    # Otherwise, strip again and respect subscenarios (see above)
                    else:
                        results = load_file(path)
                        rv[s1][s2][day] = {}
                        for k in results.keys():
                            if not is_subscenario(scenario, subscenario, k):
                                continue
                            if type(strip) is list:
                                rv[s1][s2][day][k] = {s: results[k][s] for s in strip}
                            else:
                                rv[s1][s2][day][k] = {strip: results[k][strip]}
        # Return the assembled results
        return rv

The following functions are used to generate False Accept Rates (FARs) and False Reject Rates (FRRs) for the datasets.

In [ ]:
# Calculate False Accept and False Reject Rate
def gen_far_frr(scenario, data, param, minv=None, maxv=None, increments=1000, filter_func=None, threshold=None):
    """Generate a bunch of statistics on the sensor pairs to allow calculation of error rates, overlap, etc.
    
    :param scenario: The scenario (S_CAR, S_OFFICE)
    :param data: The data to operate on, as loaded by load_audio_feature
    :param param: The parameter to operate on (e.g., max_xcorr)
    :param minv: The minimum value to use for the threshold search
    :param maxv: The maximum value to use for the threshold search
    :param increments: The number of increments the area between minv and maxv should be divided
    :param filter_func: A function to apply to all data before further processing, to exclude specific samples
        that do not meet some criteria (e.g., energy too low to consider, ...)
    :param threshold: Use a fixed threshold for computations instead of finding your own between minv and maxv.
    :return: A dictionary containing false positive, false negative, true positive and true negative counts for
        each considered threshold and pair of sensors."""
    # Histogram intersection function adapted from
    # http://blog.datadive.net/histogram-intersection-for-change-detection/
    def histogram_intersection(h1, h2, bins):
        bins = np.diff(bins)
        sm = 0
        for i in range(len(bins)):
            sm += min(bins[i]*h1[i], bins[i]*h2[i])
        return sm
    
    # Ensure the scenario is sane
    assert scenario in [S_CAR, S_OFFICE, S_MOBILE]
    # Sanity check that the provided data makes sense
    if (minv is not None and maxv is None) or (minv is None and maxv is not None):
        raise Exception("Need to provide either both or none of minv, maxv")
    if (minv is not None and maxv is not None and threshold is not None):
        raise Exception("Do not provide both a fixed threshold and an area for a threshold search.")
    
    # Prepare a few variables
    filter_res = None
    colo = []
    ncolo = []
    
    # Apply filter function. This allows us to enforce certain requirements on the data
    # (e.g., minimum energy, ...)
    if filter_func is not None:
        data, filter_res = filter_func(data, scenario)
        print("Filter result:")
        pprint(filter_res)
    res = {}
    
    # Find the range (minimum and maximum values) to try for the threshold, based on the data
    if (minv is None or maxv is None) or threshold is not None:
        #print("Finding min and max...")
        minv = 100000.0
        maxv = 0.0
        if scenario == S_CAR or scenario == S_MOBILE:
            for s1 in data:
                for s2 in data[s1]:
                    for d in data[s1][s2]:
                        minv = min(data[s1][s2][d][param], minv)
                        maxv = max(data[s1][s2][d][param], maxv)
        if scenario == S_OFFICE:
            for s1 in data:
                for s2 in data[s1]:
                    for day in data[s1][s2]:
                        for d in data[s1][s2][day]:
                            minv = min(data[s1][s2][day][d][param], minv)
                            maxv = max(data[s1][s2][day][d][param], maxv)
        print("min: %s, max: %s" % (minv, maxv))
        #print("Threshold fpr, fnr, tpr, tnr")
        
        if 0.0 <= minv <= 1.0 and 0.0 <= maxv <= 1.0:
            minv = 0.0
            maxv = 1.0
        elif 0.0 <= minv <= 100.0 and 0.0 <= maxv <= 100.0:
            minv = 0.0
            maxv = 100.0
        else:
            raise Exception("Not implemented")

    # Determine error counts for all considered thresholds
    if threshold is None:
        # Try the requested number of increments between minv and maxv
        for i in range(increments+1):
            # Calculate current threshold for this increment
            threshold = minv + i * ((maxv - minv) / increments)
            
            res[threshold] = {}
            
            # If we are looking at data from the car scenario...
            if scenario == S_CAR or scenario == S_MOBILE:
                # ... iterate through all pairs of sensors
                for s1 in data:
                    res[threshold][s1] = {}
                    for s2 in data[s1]:
                        res[threshold][s1][s2] = {
                            "ta": 0.0,  # True accept
                            "tr": 0.0,  # True reject
                            "fa": 0.0,  # False accept
                            "fr": 0.0   # False reject
                        }
                        # Iterate through all readings for this pair
                        for d in data[s1][s2]:
                            # If the reading is above the threshold, accept it, otherwise reject
                            accept = data[s1][s2][d][param] >= threshold
                            # Check if we accepted correctly or incorrectly and track for error rates
                            if is_colocated(s1, s2, scenario, d):
                                if i == 0:
                                    colo.append(data[s1][s2][d][param])
                                if accept:
                                    res[threshold][s1][s2]["ta"] += 1
                                    # true_acc += 1
                                else:
                                    res[threshold][s1][s2]["fr"] += 1
                                    # false_rej += 1
                            else:
                                if i == 0:
                                    ncolo.append(data[s1][s2][d][param])
                                if accept:
                                    res[threshold][s1][s2]["fa"] += 1
                                    # false_acc += 1
                                else:
                                    res[threshold][s1][s2]["tr"] += 1
                                    # true_rej += 1
            
            # Do the same for the office (slightly more complicated due to different data structure)
            if scenario == S_OFFICE:
                for s1 in data:
                    res[threshold][s1] = {}
                    for s2 in data[s1]:
                        res[threshold][s1][s2] = {
                            "ta": 0.0,  # True accept
                            "tr": 0.0,  # True reject
                            "fa": 0.0,  # False accept
                            "fr": 0.0   # False reject
                        }
                        for day in data[s1][s2]:
                            for d in data[s1][s2][day]:
                                accept = data[s1][s2][day][d][param] >= threshold
                                if is_colocated(s1, s2, scenario):
                                    if i == 0:
                                        colo.append(data[s1][s2][day][d][param])
                                    if accept:
                                        res[threshold][s1][s2]["ta"] += 1
                                        # true_acc += 1
                                    else:
                                        res[threshold][s1][s2]["fr"] += 1
                                        # false_rej += 1
                                else:
                                    if i == 0:
                                        ncolo.append(data[s1][s2][day][d][param])
                                    if accept:
                                        res[threshold][s1][s2]["fa"] += 1
                                        # false_acc += 1
                                    else:
                                        res[threshold][s1][s2]["tr"] += 1
                                        # true_rej += 1
    
    else:
        # A fixed threshold has been given. Only check this threshold
        true_acc = 0.0
        false_acc = 0.0
        true_rej = 0.0
        false_rej = 0.0
    
        # Go through everything as before (see above), but this time do not generate
        # a result dictionary. Instead, we just want to compute the error rates for
        # the specified threshold and print them.
        if scenario == S_CAR or scenario == S_MOBILE:
            for s1 in data:
                for s2 in data[s1]:
                    for d in data[s1][s2]:
                        accept = data[s1][s2][d][param] >= threshold
                        if is_colocated(s1, s2, scenario, d):
                            if accept:
                                true_acc += 1
                            else:
                                false_rej += 1
                        else:
                            if accept:
                                false_acc += 1
                            else:
                                true_rej += 1
        if scenario == S_OFFICE:
            for s1 in data:
                for s2 in data[s1]:
                    for day in data[s1][s2]:
                        for d in data[s1][s2][day]:
                            accept = data[s1][s2][day][d][param] >= threshold
                            if is_colocated(s1, s2, scenario):
                                if accept:
                                    true_acc += 1
                                else:
                                    false_rej += 1
                            else:
                                if accept:
                                    false_acc += 1
                                else:
                                    true_rej += 1

        # Calculate error rates
        fpr = false_acc / (false_acc + true_rej)
        fnr = false_rej / (false_rej + true_acc)
        tpr = true_acc / (true_acc + false_rej)
        tnr = true_rej / (true_rej + false_acc)
        #print(threshold, fpr, fnr, tpr, tnr)
        # print("far", fpr, "frr", fnr)
        return {"far": fpr, "frr": fnr, "threshold": threshold}
    
    # Plot the overlap between the classes
    plot_distributions(data, scenario, param, minv, maxv)
    
    # Also calculate overlap as a number
    hc, bins = np.histogram(colo, np.arange(minv, maxv, maxv/100.0), density=True)
    hn, _ = np.histogram(ncolo, np.arange(minv, maxv, maxv/100.0), density=True)
    print("Intersection:", histogram_intersection(hc, hn, bins))

    return res


def frr_for_far(data, target_far, sources=None, targets=None):
    """Calculate the False Reject Rate (FRR) implied by a given target False Accept Rate (FAR).
    Can also compute this for subsets of the datasets, given by "sources" and "targets". In this
    case, it will consider all combinations of sources and targets (e.g., if sources = ["1"] and
    targets = ["2", "3"], it will consider 1-2 and 1-3, but not 2-3).
    
    :param data: The data as a dictionary, as produced by gen_far_frr or the import functions.
    :param target_far: The false accept rate to aim for. Note that 1.0 implies 100%, so 0.1% should
        be written as 0.001.
    :param sources: A list of sensors (as unpadded strings) to use as sources, or None if all
        should be considered.
    :param targets: A list of sensors (as unpadded strings) to use as targets, or NOne if all
        should be considered
    :return: A 3-tuple of observed FAR, FRR, and the used threshold.
    """
    def is_source(sensor):
        if sources is None:
            return True
        else:
            return str(sensor) in sources
    
    def is_target(sensor):
        if targets is None:
            return True
        else:
            return str(sensor) in targets

    # Initialize previous values with bogus values to ensure they are never used in the first iteration.
    prev_far = -500.0
    prev_frr = -500.0
    prev_threshold = -500.0
    for threshold in data:
        false_acc = 0.0
        false_rej = 0.0
        true_acc = 0.0
        true_rej = 0.0
        for s1 in data[threshold]:
            # Check if sensor is supposed to be included
            if not (is_source(s1) or is_target(s1)):
                continue
            for s2 in data[threshold][s1]:
                # Check if sensor pairing is supposed to be included
                if not ((is_source(s1) and is_target(s2)) or (is_source(s2) and is_target(s1))):
                    continue
                # Include the numbers in the overall count
                false_acc += data[threshold][s1][s2]['fa']
                false_rej += data[threshold][s1][s2]['fr']
                true_acc += data[threshold][s1][s2]['ta']
                true_rej += data[threshold][s1][s2]['tr']
        
        # Calculate error rates
        far = false_acc / (false_acc + true_rej)
        frr = false_rej / (false_rej + true_acc)
        
        if far > target_far:
            # The computed FAR is above the target FAR. Save current values and carry on
            prev_far = far
            prev_frr = frr
            prev_threshold = threshold
        else:
            # We have reached or passed the target FAR. Determine if the previous value was a better fit
            if abs(target_far - far) < abs(target_far - prev_far):
                # We are closer to the target FAR than the previous FAR, use our values
                return (far, frr, threshold)
            else:
                # The previous values were closer, use them
                return (prev_far, prev_frr, prev_threshold)
    assert False, "This statement should never be reached. Last error rates: FAR " + str(prev_far) + ", FRR " + str(prev_frr)

    
def error_rate_for_threshold(data, threshold):
    """Calculate the error rates when using a specific threshold.
    
    :param data: The data, as generated by gen_far_frr
    :param threshold: The threshold to generate the error rates for
    :return: A 2-tuple of far, frr
    """
    false_acc = 0.0
    false_rej = 0.0
    true_acc = 0.0
    true_rej = 0.0
    for s1 in data[threshold]:
        for s2 in data[threshold][s1]:
            # Include the numbers in the overall count
            false_acc += data[threshold][s1][s2]['fa']
            false_rej += data[threshold][s1][s2]['fr']
            true_acc += data[threshold][s1][s2]['ta']
            true_rej += data[threshold][s1][s2]['tr']

    # Calculate error rates
    far = false_acc / (false_acc + true_rej)
    frr = false_rej / (false_rej + true_acc)
        
    return (far, frr) 

Functions for saving and loading results, and their helper functions.

In [ ]:
# Save the resulting JSON data from the get_far_frr function to a file
def save_result_json(data, paper, interval, scenario, modality, subscenario=None, suffix=None):
    """Save a dictionary into a file as JSON. Mostly used for cache files.
    
    :param data: The data to save.
    :param paper: The paper to save it under (SOUNDPROOF, ...)
    :param interval: The interval to save it under
    :param scenario: The scenario (S_CAR, S_OFFICE)
    :param modality: The modality (max_xcorr, ...)
    :param subscenario: The subscenario, or None if global.
    :param suffix: A suffix to place before the .json"""
    path = '/'.join([PREFIX_JSON, scenario, paper, modality])
    filename = path + '/' + interval
    if subscenario is not None:
        filename += '-' + subscenario
    if suffix is not None:
        filename += '_' + suffix
    filename += '.json'
    makedirs(path, exist_ok=True)
    with open(filename, 'w') as fo:
        json.dump(data, fo, separators=(',', ': '), indent=4)

# Check if a result cache file (generated by save_result_json) exists
def result_exists(paper, interval, scenario, modality, subscenario=None, suffix=None):
    """Check if a cache file for a set of parameters exists.
    
    :param paper: The paper (SOUNDPROOF, ...)
    :param interval: The interval
    :param scenario: The scenario (S_CAR, S_OFFICE)
    :param modality: The modality (max_xcorr, ...)
    :param subscenario: The subscenario, or None if global.
    :return: True if a file exists, otherwise False"""
    path = '/'.join([PREFIX_JSON, scenario, paper, modality])
    filename = path + '/' + interval
    if subscenario is not None:
        filename += '-' + subscenario
    if suffix is not None:
        filename += '_' + suffix
    filename += '.json'
    return isfile(filename)

# Load result cache file (generated by save_result_json)
def load_result(paper, interval, scenario, modality, subscenario=None, suffix=None):
    """Check if a cache file for a set of parameters exists.
    
    :param paper: The paper(SOUNDPROOF, ...)
    :param interval: The interval
    :param scenario: The scenario (S_CAR, S_OFFICE)
    :param modality: The modality (max_xcorr, ...)
    :param subscenario: The subscenario, or None if global.
    :return: The loaded cache as a dictionary"""
    assert result_exists(paper, interval, scenario, modality, subscenario, suffix)
    rv = {}
    
    # Construct file name
    path = '/'.join([PREFIX_JSON, scenario, paper, modality])
    filename = path + '/' + interval
    if subscenario is not None:
        filename += '-' + subscenario
    if suffix is not None:
        filename += '_' + suffix
    filename += '.json'
    
    # Load the data
    with open(filename, 'r') as fo:
        r = json.load(fo)
    
    # Cast thresholds to float, if necessary
    if suffix is None:
        for threshold in r:
            rv[float(threshold)] = r[threshold]
        return rv
    else:
        return r

Visualization helper functions

In [ ]:
# Visualize FAR and FRR
def plot_far_frr(result_data, paper=None, scenario=None, subscenario=None, modality=None, feature=None, interval=None, save=False, xlow=0.0, xhigh=1.0):
    """Generate the FAR and FRR for specific results data, and determine and plot the EER.
    
    :param result_data: data in the format generated by gen_far_frr or the import functions
    :param xlow: lower limit for the threshold search
    :param xhigh: upper limit for the threshold search
    :return: A 3-tuple (threshold, fpr, fnr)
    """
    # We are getting a fairly complex input, so we first have to generate a simpler data structure from it
    if save:
        assert paper is not None
        assert scenario is not None
        assert interval is not None
        assert modality is not None
        assert feature is not None
        filename = '/'.join([PREFIX_PLOTS, scenario, paper, modality])
        if subscenario is not None:
            filename += '/' + '-'.join([feature, interval, subscenario, 'error-rates']) + '.eps'
        else:
            filename += '/' + '-'.join([feature, interval, 'error-rates']) + '.eps'
    results = {}
    for threshold in result_data:
        # Prepare result and temporary variables
        results[threshold] = {}
        true_acc = 0.0
        true_rej = 0.0
        false_acc = 0.0
        false_rej = 0.0
        
        # Load counts into the temporary vars
        for s1 in result_data[threshold]:
            for s2 in result_data[threshold][s1]:
                true_acc += result_data[threshold][s1][s2]["ta"]
                true_rej += result_data[threshold][s1][s2]["tr"]
                false_acc += result_data[threshold][s1][s2]["fa"]
                false_rej += result_data[threshold][s1][s2]["fr"]
        
        # Calculate error rates
        # False Accept Rate (FAR)
        fpr = false_acc / (false_acc + true_rej)
        # False Reject Rate (FRR)
        fnr = false_rej / (false_rej + true_acc)
        # True Accept Rate (TAR)
        tpr = true_acc / (true_acc + false_rej)
        # True Reject Rate (TRR)
        tnr = true_rej / (true_rej + false_acc)

        # Put them in a data structure the visualization function understands
        results[threshold] = {"fpr": fpr, "fnr": fnr, "tpr": tpr, "tnr": tnr}
    
    # Actually do the plotting of the error rates
    fig, ax = plt.subplots()
    ax.plot(sorted(results.keys()), [results[threshold]["fpr"] for threshold in sorted(results.keys())], '--', label='FAR')
    ax.plot(sorted(results.keys()), [results[threshold]["fnr"] for threshold in sorted(results.keys())], label='FRR')
    
    # Determine the point where FAR and FRR are closest (i.e., the Equal Error Rate - EER)
    for threshold in sorted(results.keys()):
        if results[threshold]["fpr"] <= results[threshold]["fnr"]:
            fpr = results[threshold]["fpr"]
            fnr = results[threshold]["fnr"]
            if abs(prev_fpr - prev_fnr) < abs(results[threshold]["fpr"] - results[threshold]["fnr"]):
                fpr = prev_fpr
                fnr = prev_fnr
                threshold = prev_thres
            print("Thresh", threshold, "FAR", fpr, "FRR", fnr)
            #ax.plot([threshold, threshold], [0.0, fnr], 'k-')
            #ax.plot([0.0, threshold], [fnr, fnr], 'k-')
            ax.set(xlabel='Similarity Threshold', ylabel='Error Rate', xlim=(xlow,xhigh))
            ax.legend()
            if save:
                plt.savefig(filename, format='eps', dpi=1000)
            plt.show()

            return fpr, fnr, threshold
        prev_fpr = results[threshold]["fpr"]
        prev_fnr = results[threshold]["fnr"]
        prev_thres = threshold


def prune_results(result_data, prune):
    """Remove specific sensors from the results
    
    :param result_data: Result data, as formatted by gen_far_frr or the result import functions
    :param prune: A list of dictionary keys (strings) to remove. Will remove any pair that involves at least
        one of the provided keys (so if "01" should be removed, it will remove X->01 and 01->X for all keys X)
    :return: the pruned result dataset
    """
    # Make a deep copy of the dictionary, because we don't want to modify the original
    result = copy.deepcopy(result_data)
    # Go through the dictionary looking for keys to remove
    for t in result:
        # Ensure that we are not using an internal iterator on the dictionary, since we are about to modify the
        # dictionary during iteration
        for s1 in list(result[t].keys()):
            if s1 in prune:
                del(result[t][s1])
            else:
                # Ditto here
                for s2 in list(result[t][s1]):
                    if s2 in prune:
                        del([result[t][s1][s2]])
    return result


def pair_error_rate(result_data, s1, s2, threshold):
    """Determine the error rate between sensors s1 and s2.
    
    :param result_data: Result data, as formatted by gen_far_frr or the result import functions
    :param s1: The first sensor (string)
    :param s2: The second sensor (string)
    :param threshold: The threshold to use
    :return: 4-tuple of FAR, FRR, TAR, TRR.
    """
    # Ensure dataset meets expectations
    assert threshold in result_data
    if int(s1) > int(s2):
        t = s1
        s1 = s2
        s2 = t
    assert s1 in result_data[threshold]
    assert s2 in result_data[threshold][s1]
    
    # Collect statistics
    true_acc = result_data[threshold][s1][s2]["ta"]
    true_rej = result_data[threshold][s1][s2]["tr"]
    false_acc = result_data[threshold][s1][s2]["fa"]
    false_rej = result_data[threshold][s1][s2]["fr"]
    
    # Calculate error rates
    # False Accept Rate (FAR)
    if false_acc + true_rej > 0:
        fpr = false_acc / (false_acc + true_rej)
    else:
        fpr = None

    # False Reject Rate (FRR)
    if false_rej + true_acc > 0:
        fnr = false_rej / (false_rej + true_acc)
    else:
        fnr = None

    # True Accept Rate (TAR)
    if true_acc + false_rej > 0:
        tpr = true_acc / (true_acc + false_rej)
    else:
        tpr = None
    
    # True Reject Rate (TRR)
    if true_rej + false_acc > 0:
        tnr = true_rej / (true_rej + false_acc)
    else:
        tnr = None
    
    return (fpr, fnr, tpr, tnr)
    
    
def plot_distributions(data, scenario, param, minv, maxv, legend=True, save=None):
    """Plot the distribution of values to see how far they overlap.
    
    :param data: The data to plot.
    :param scenario: The scenario (S_CAR, S_OFFICE)
    :param param: The parameter to look into (max_xcorrm ...)
    :param minv: The minimum on the x-axis to show
    :param maxv: The maximum on the x-axis to show
    :param legend: Boolean indicating whether a legend should be printed
    :param save: A path under which the plot should be saved, or None if it should not be saved."""
    colo = []
    ncolo = []
    if scenario == S_CAR or scenario == S_MOBILE:
        for s1 in data:
            for s2 in data[s1]:
                for d in data[s1][s2]:
                    if is_colocated(s1, s2, scenario, d):
                        colo.append(data[s1][s2][d][param])
                    else:
                        ncolo.append(data[s1][s2][d][param])
    else:
        for s1 in data:
            for s2 in data[s1]:
                if is_colocated(s1, s2, scenario):
                    for day in data[s1][s2]:
                        for d in data[s1][s2][day]:
                            if not math.isnan(data[s1][s2][day][d][param]):
                                colo.append(data[s1][s2][day][d][param])
                else:
                    for day in data[s1][s2]:
                        for d in data[s1][s2][day]:
                            if not math.isnan(data[s1][s2][day][d][param]):
                                ncolo.append(data[s1][s2][day][d][param])
    plt.figure()
    sns.kdeplot(colo, label="Colocated")
    ax = sns.kdeplot(ncolo, label="Non-colocated", linestyle="--")
    # ax.set_xlabel("Similarity Percentage", fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    if legend:
        plt.legend(prop={'size': 14})
    if not legend:
        ax.legend().set_visible(False)
    if 0.0 <= minv <= 1.0 and 0.0 <= maxv <= 1.0:
        minlim = 0.0
        maxlim = 1.0
    elif 0.0 <= minv <= 100.0 and 0.0 <= maxv <= 100.0:
        minlim = 0.0
        maxlim = 100.0
    else:
        raise Exception("Not implemented")
    ax.set_xlim(minlim,maxlim)
    if save is not None:
        plt.savefig(save, format='eps', dpi=1000)
    plt.show()

# Data analysis
At this point, we have defined all functions that we need for data analysis.

To work with the FAR and FRR data, we need to first compute a series of accept and reject results for specific thresholds on the data. **This is an expensive operation** (on the order of up to a few days for large datasets like the office experiment), but it only needs to be run once - the results are cached on disk for future operations.

If you have obtained this code together with the dataset, it should already contain these caches, no need to regenerate them unless you want to reproduce our results.

In [ ]:
SUB_LIST = {
    S_CAR: [SU_PARKED, SU_CITY, SU_HIGHWAY, None],
    S_OFFICE: [SU_WDAY, SU_NIGHT, SU_WEND, None],
    S_MOBILE: [None]
}
for scenario in [S_CAR, S_OFFICE, S_MOBILE]:
    for interval in reversed(INTERVALS):
        for subscenario in SUB_LIST[scenario]:
            print(scenario, subscenario, interval)
            # Load the data
            data = load_audio_feature(AUDIO_FP, interval, scenario, strip='fingerprints_similarity_percent', subscenario=subscenario)
            # Compute the FAR and FRR for a set of different thresholds
            rv = gen_far_frr(scenario, data, 'fingerprints_similarity_percent', minv=0.0, maxv=100.0, increments=3000)
            # Save the resulting caches
            save_result_json(rv, AUDIO_FP, interval, scenario, 'fingerprints_similarity_percent', subscenario=subscenario)

Now that we have generated the caches, we can generate the actual results. This includes the actual error rates, the Equal Error Rates (EERs), and a few more statistics on all scenarios and subscenarios with all interval sizes. It also computes the robustness data (i.e, applying the EER threshold from one scenario on another dataset and seeing what the resulting error rates are).

In [ ]:
SCENARIO_SET = set([S_CAR, S_OFFICE, S_MOBILE])
SUB_LIST = {
    S_CAR: [SU_PARKED, SU_CITY, SU_HIGHWAY, None],
    S_OFFICE: [SU_WDAY, SU_NIGHT, SU_WEND, None],
    S_MOBILE: [None]
}


SUB_SETS = {
    S_CAR: set([SU_PARKED, SU_CITY, SU_HIGHWAY]),
    S_OFFICE: set([SU_WDAY, SU_NIGHT, SU_WEND]),
    S_MOBILE: set([None])
}

robustness_output = {}

# We go through every interval...
for interval in reversed(INTERVALS):
    thresholds = {}
    # ...and every scenario
    for scenario in SCENARIO_SET:
        thresholds[scenario] = {}
        # ...and every subscenario (including "None", which is the full scenario)
        for subscenario in SUB_LIST[scenario]:
            thresholds[scenario][subscenario] = {}
            print(scenario, subscenario, interval)
            # Prepare output JSON
            result = {"base": {}, "adversarial": {}}

            # We have cached results, load them
            print("Using cached results...")
            rv = load_result(AUDIO_FP, interval, scenario, 'fingerprints_similarity_percent', subscenario=subscenario)
            
            # Plot the error rates and determine the EER
            far, frr, thresh = plot_far_frr(rv, xlow=0.0, xhigh=100.0, paper=AUDIO_FP, scenario=scenario, subscenario=subscenario, modality='audio', feature='fingerprints_similarity_percent', interval=interval)
            result["base"]["eer"] = {"far": far, "frr": frr, "threshold": thresh}
            thresholds[scenario][subscenario]["eer"] = {"threshold": thresh, "far": far, "frr": frr}
            
            # Try a number of target false accept rates to determine the respective FRR and threshold
            for target_far in [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05]:
                far, frr, threshold = frr_for_far(rv, target_far)
                result["base"]["far_%s" % target_far] = {"far": far, "frr": frr, "threshold": threshold}
                thresholds[scenario][subscenario]["far_%s" % target_far] = {"threshold": threshold, "far": far, "frr": frr}
            
            # Save the result
            save_result_json(result, AUDIO_FP, interval, scenario, 'fingerprints_similarity_percent', subscenario=subscenario, suffix="rates")
            
            print("")
            print("")
    # At this point, the thresholds dictionary should contain information for all scenarios and subscenarios
    # We will now perform robustness checks.  This means that we are interested in how well the performance
    # holds up if we start using the optimal threshold from one scenario on the other scenario, and the same
    # for subscenarios within each scenario.  This is written less efficiently than it could be, in the interest
    # of simpler code - loading all result caches twice does not have a massive impact on the performance.
    print("Generating robustness data...")
    # Again, go through all scenarios and subscenarios
    for scenario in SCENARIO_SET:
        result = {scenario: {}}
        for subscenario in SUB_LIST[scenario]:
            
            error_rates = ["far_%s" % rate for rate in [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05]]
            error_rates.append("eer")
            
            # We have cached results, load them
            rv = load_result(AUDIO_FP, interval, scenario, 'fingerprints_similarity_percent', subscenario=subscenario)
            
            if subscenario is None:
                # This is a "global" result (no specific subscenario) - compare with other scenario
                for target in SCENARIO_SET - set([scenario]):
                    # Prepare output
                    result[scenario][target] = {}
                    
                    # Go through all target error rates (see above)
                    for error_rate in error_rates:
                        # Load the threshold from the target scenarios
                        threshold = thresholds[target][None][error_rate]["threshold"]
                        orig_far = thresholds[scenario][None][error_rate]["far"]
                        orig_frr = thresholds[scenario][None][error_rate]["frr"]

                        # Determine the error rate the scenario would have with the threshold from the target scenario
                        far, frr = error_rate_for_threshold(rv, threshold)
                        # ...and save it.
                        result[scenario][target][error_rate] = {
                            "threshold": threshold, 
                            "far": far, 
                            "frr": frr,
                        }
                        
                        # We also want to print out a formatted robustness table, but only for the EER threshold
                        if error_rate == "eer":
                            # Save the old error rates
                            result[scenario][target][error_rate]["orig_far"] = orig_far
                            result[scenario][target][error_rate]["orig_frr"] = orig_frr
                            
                            # Ensure the structure of the robustness_output dictionary will support our actions
                            if scenario not in robustness_output:
                                robustness_output[scenario] = {target: {}}
                            if target not in robustness_output[scenario]:
                                robustness_output[scenario][target] = {}
                                
                            # Save the robustness information
                            robustness_output[scenario][target][interval] = {
                                "far": far, 
                                "frr": frr,
                                "orig_far": orig_far,
                                "orig_frr": orig_frr,
                                "far_change_abs": orig_far - far,
                                "frr_change_abs": orig_frr - frr,
                                "total_change_abs": abs(orig_far - far) + abs(orig_frr - frr),
                                "far_change_rel": (orig_far / far) * 100,
                                "frr_change_rel": (orig_frr / frr) * 100
                            }
                
            else:  # Subscenario is not None
                result[subscenario] = {}
                # Iterate through all other subscenarios
                for target_ss in SUB_SETS[scenario] - set([subscenario]):
                    result[subscenario][target_ss] = {}
                    # Go through all error rates
                    for error_rate in error_rates:
                        # The following process is identical to the one above, see there for documentation
                        threshold = thresholds[scenario][target_ss][error_rate]["threshold"]
                        orig_far = thresholds[scenario][subscenario][error_rate]["far"]
                        orig_frr = thresholds[scenario][subscenario][error_rate]["frr"]
                        
                        far, frr = error_rate_for_threshold(rv, threshold)
                        result[subscenario][target_ss][error_rate] = {
                            "threshold": threshold, 
                            "far": far, 
                            "frr": frr,
                        }
                        if error_rate == "eer":
                            result[subscenario][target_ss][error_rate]["orig_far"] = orig_far
                            result[subscenario][target_ss][error_rate]["orig_frr"] = orig_frr
                            
                            if scenario not in robustness_output:
                                robustness_output[scenario] = {}
                            if subscenario not in robustness_output[scenario]:
                                robustness_output[scenario][subscenario] = {}
                            if target_ss not in robustness_output[scenario][subscenario]:
                                robustness_output[scenario][subscenario][target_ss] = {}
                            robustness_output[scenario][subscenario][target_ss][interval] = {
                                "far": far, 
                                "frr": frr,
                                "orig_far": orig_far,
                                "orig_frr": orig_frr,
                                "far_change_abs": orig_far - far,
                                "frr_change_abs": orig_frr - frr,
                                "total_change_abs": abs(orig_far - far) + abs(orig_frr - frr),
                                "far_change_rel": (orig_far / far) * 100,
                                "frr_change_rel": (orig_frr / frr) * 100
                            }

        # Save the results to a JSON file
        save_result_json(result, AUDIO_FP, interval, scenario, 'fingerprints_similarity_percent', subscenario="robustness", suffix="summary")

        
# Okay, now it's time to print out a table of the robustness data
# Once again, we go through all pairs of scenarios:
for scenario in SCENARIO_SET:
    for scenario2 in SCENARIO_SET - set([scenario]):
        # Print out table header
        # "scenario" is the used dataset, scenario2 is the one whose threshold was used
        print("Robustness", scenario, scenario2)
        # What do these abbreviations mean?
        # - Int = Interval that was used
        # - FAR and FRR = Obtained false accept / reject rates
        # - ofar and ofrr = Original FAR and FRR from the base scenario, for comparison
        # - sprd = spread between FAR and FRR, i.e. abs(FAR - FRR)
        # - osprd = Spread between original FAR and FRR, i.e. abs(ofar - ofrr)
        # - aca and rca = Absolute change in false accept / reject rate, i.e. ofar - far
        # - tca = Absolute changes summed up, i.e. aca + rca
        # - acr and rcr = Relative changes in FAR and FRR, i.e. (ofar / far) * 100
        # - oeer = original EER
        # - eer = new EER, i.e. (far + frr) / 2.0
        # - eerabs = absolute change in EER
        print("Int", "FAR", "FRR", "ofar", "ofrr", "sprd", "osprd", "aca", "rca", "tca", "acr", "rcr", "oeer", "eer", "eerabs", sep='\t|')
        print("-" + "-------+"*14 + "-------")
        scenpair = robustness_output[scenario][scenario2]
        for interval in INTERVALS:
            oeer = (scenpair[interval]['orig_far'] + scenpair[interval]['orig_frr']) / 2.0
            eer = (scenpair[interval]['far'] + scenpair[interval]['frr']) / 2.0
            ospread = abs(scenpair[interval]['orig_far'] - scenpair[interval]['orig_frr'])
            spread = abs(scenpair[interval]['far'] - scenpair[interval]['frr'])
            print(interval, 
                  round(scenpair[interval]['far'], 3), 
                  round(scenpair[interval]['frr'], 3), 
                  round(scenpair[interval]['orig_far'], 3), 
                  round(scenpair[interval]['orig_frr'], 3), 
                  round(spread, 3),
                  round(ospread, 3),
                  round(scenpair[interval]['far_change_abs'], 3), 
                  round(scenpair[interval]['frr_change_abs'], 3),
                  round(scenpair[interval]['total_change_abs'], 3),
                  round(scenpair[interval]['far_change_rel'], 1), 
                  round(scenpair[interval]['frr_change_rel'], 1),
                  round(oeer, 3),
                  round(eer, 3),
                  round(oeer - eer, 3),
                  sep='\t|')
        print("")

print("\n\n")
# Print the same table for all combinations of subscenarios
# Process is the same as above.
for scenario in SCENARIO_SET:
    for subscenario in SUB_LIST[scenario]:
        for target_ss in SUB_LIST[scenario]:
            if target_ss == subscenario or target_ss is None or subscenario is None:
                continue
            print("Robustness", scenario, subscenario, target_ss)
            print("Int", "FAR", "FRR", "ofar", "ofrr", "sprd", "osprd", "aca", "rca", "tca", "acr", "rcr", "oeer", "eer", "eerabs", sep='\t|')
            print("-" + "-------+"*14 + "-------")
            scenpair = robustness_output[scenario][subscenario][target_ss]
            for interval in INTERVALS:
                oeer = (scenpair[interval]['orig_far'] + scenpair[interval]['orig_frr']) / 2.0
                eer = (scenpair[interval]['far'] + scenpair[interval]['frr']) / 2.0
                ospread = abs(scenpair[interval]['orig_far'] - scenpair[interval]['orig_frr'])
                spread = abs(scenpair[interval]['far'] - scenpair[interval]['frr'])
                print(interval, 
                      round(scenpair[interval]['far'], 3), 
                      round(scenpair[interval]['frr'], 3), 
                      round(scenpair[interval]['orig_far'], 3), 
                      round(scenpair[interval]['orig_frr'], 3),
                      round(spread, 3),
                      round(ospread, 3),
                      round(scenpair[interval]['far_change_abs'], 3), 
                      round(scenpair[interval]['frr_change_abs'], 3),
                      round(scenpair[interval]['total_change_abs'], 3),
                      round(scenpair[interval]['far_change_rel'], 1), 
                      round(scenpair[interval]['frr_change_rel'], 1),
                      round(oeer, 3),
                      round(eer, 3),
                      round(oeer - eer, 3),
                      sep='\t|')
            print("")

# Randomness
To evaluate the randomness of the generated fingerprints, we use techniques developed by Brüsch et al. in their paper ["On the Secrecy of Publicly Observable Biometric Features: Security Properties of Gait for Mobile Device Pairing" (CoRR abs/1804.03997)](https://arxiv.org/abs/1804.03997).

In [ ]:
# This code is based on code by Arne Brüsch, cf. https://github.com/abruesch/randomness-figures
# It has been adapted to fit our use case. For the details on the original concept, see:
# Arne Brüsch, Ngu Nguyen, Dominik Schürmann, Stephan Sigg, and Lars Wolf. 2018. 
# On the Secrecy of Publicly Observable Biometric Features: Security Properties of Gait for 
# Mobile Device Pairing. CoRR abs/1804.03997 (2018). https://arxiv.org/abs/1804.03997
# The original code is licensed under the GPLv3.

import sys
import random
import os

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy.misc import comb
from scipy.special import binom
from cycler import cycler


def random_walk(key, sum_distribution, transition_count, transition_probs, bitlength=128):
    ''' takes string of '0' and '1' and turns them into random walks in a galton board. Effectively computes
    the cumulative sums distribution for every prefix of the input string. '''

    transition_count[0] += 1
    ret = [0]
    for i, b in enumerate(key[:bitlength]):
        val = -1
        if b == '1':
            val = 1
            transition_probs[i] += 1
        ret.append(ret[-1] + val)

    sum_distribution.append(ret[-1])
    return (sum_distribution, transition_count, transition_probs)


def markov_transitions(transition_probs, transition_count, bits=128):
    ''' transition probabilities from every nth to (n+1)th bit. '''
    norm_transition_probs = [transition_probs[x] / transition_count[0]
                             for x in range(0, bits)]
    plt.clf()
    plt.xlabel('nth bit')
    plt.ylabel('Probability for 1')
    plt.ylim([0, 1])
    plt.xlim([0, bits])
    plt.rcParams.update({'font.size': 18})
    plt.plot(norm_transition_probs, color='b')
    plt.plot([0.0, bits], [0.5, 0.5], 'k:')
    plt.show()
    print("Markov:", np.median(norm_transition_probs))

from scipy.misc import comb

def binomial_plot(n=256):
    ''' Theoretical binomial distribution that is plotted as red line into figure.'''
    x = np.arange(n)
    # y = list(map(lambda xi: comb(n,xi)/2**n, x))
    y = list(binom(n,x)*0.5**x*(0.5)**(n-x))
    x = list(map(lambda r: r-n/2, x))
    plt.plot(x,y,color='r')

#def binomial_plot(bins,n):
#    variance =  n * 0.5 * 0.5
#    sigma = np.sqrt(variance)
#    y = mlab.normpdf(np.asarray(list(range(-128,128))), 0, sigma)
#    plt.plot(np.asarray(list(range(-128,128))),y,color='r')

def distribution(sum_distribution, bits=128, dist_xlim=None, save_to=None):
    ''' Plots the cumulative sums distribution and saves figure. '''
    plt.clf()
    if dist_xlim is not None:
        plt.xlim(dist_xlim)
    else:
        plt.xlim([-bits, bits])
    count, bins, ignored = plt.hist(sum_distribution, color='#007a9b', range=(-bits,bits), normed=True,rwidth=0.5, bins=bits)
    binomial_plot(bits*2)
    # Get the current axes
    ax = plt.gca()
    # Get limits
    start, end = ax.get_xlim()
    if abs(start) > 150:
        # We are appearantly plotting a very long fingerprint, let's make the ticks more sparse
        ax.xaxis.set_ticks(np.arange(start, end+1, 100))
        
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    
    if save_to is not None:
        plt.savefig(save_to, format='eps', dpi=1000)
    plt.show()
    print("Median of distribution:", np.median(sum_distribution))
    


def plot_rand_walk(keys, bits=128, dist_xlim=None, save_distribution_to=None):
    ''' turns string consisting of '0' and '1' into random walks. While the walks are currently not plotted, the distribution
    of the cumulative sums along with the markov transitions are computed and plotted using them.'''
    sum_distribution = []
    transition_count = [0]
    transition_probs = {x: 0 for x in range(0, bits)}

    matplotlib.rcParams['axes.prop_cycle'] = cycler('color',
                                                    ['#e78a33', '#eda766', '#8d4959', '#aa7782', '#bdcd61', '#cdda89',
                                                     '#8a9c33', '#a7b566'])
    plt.rcParams.update({'font.size': 18})

    plt.ylabel('Sum')
    plt.xlabel('Keylength')

    plt.ylim([-bits, bits])
    for key in keys:
        sum_distribution, transition_count, transition_probs = random_walk(key, sum_distribution, transition_count, transition_probs, bits)
    plt.tight_layout()
    plt.rcParams.update({'font.size': 18})
    distribution(sum_distribution, bits, dist_xlim=dist_xlim, save_to=save_distribution_to)
    markov_transitions(transition_probs, transition_count, bits)


def plot_heat_map(keys, pt=plt):
    ''' creates heatmap of random walks.'''
    intensity = 1
    heatmap_array = []
    for key in keys:
        val = int(key[0])
        for i, bit in enumerate(key[1:128]):
            if bit == '1':
                val = val + 1
            elif bit == '0':
                val = val - 1
            heatmap_array.append([i, val])

    np_srt_heat = np.asarray(heatmap_array)
    X = np.take(np_srt_heat, [0], axis=1).flatten()
    Y = np.take(np_srt_heat, [1], axis=1).flatten()
    # bins = (range(max(X)), range(min(Y),max(Y)))
    bins = (range(128), range(min(Y), max(Y)))
    H, xedges, yedges = np.histogram2d(X, Y, bins=bins, normed=False)
    # H = H.T
    # H = H
    H = intensity * H
    # 'viridis'
    pt.xlabel('Sum')
    pt.ylabel('Keylength')
    pt.imshow(H, cmap='viridis', norm=matplotlib.colors.LogNorm(), interpolation='nearest', origin='upper',
              extent=[yedges[0], yedges[-1], xedges[-1], xedges[0]])
    plt.rcParams.update({'font.size': 18})
    # Plot triangle
    plus_y = [i for i in range(-1, 127)]
    plus_x = [i for i in range(1, 129)]
    minus_y = [126 - i for i in range(0, 128)]
    minus_x = [-127 + i for i in range(1, 129)]
    plt.tight_layout()
    pt.plot(plus_x, plus_y, color='r')
    pt.plot(minus_x, minus_y, color='r')


def apply_plot_heat_map(keys):
    ''' Calls plot_heat_map() and additionally saves the plotted figure.'''
    plot_heat_map(keys)
    plt.show()

We also need a couple functions to deal with fingerprints. As this was ported from a different piece of code, the constants are slightly different and need to be redefined.

In [ ]:
BASE_DIR = "/media/seemoo/data/zia-data/results/"
PREFIX_JSON='/home/seemoo/plots/json'

CAR_EXP = 'CarExp/'
OFF_EXP = 'OfficeExp/'
MOBILE_EXP = 'MobileExp/'
# Sensor mapping: car experiment
SENSORS_CAR1 = ['01', '02', '03', '04', '05', '06']
SENSORS_CAR2 = ['07', '08', '09', '10', '11', '12']
SENSORS_CAR = SENSORS_CAR1 + SENSORS_CAR2

# Sensor mapping: office experiment
SENSORS_OFFICE1 = ['01', '02', '03', '04', '05', '06', '07', '08']
SENSORS_OFFICE2 = ['09', '10', '11', '12', '13', '14', '15', '16']
SENSORS_OFFICE3 = ['17', '18', '19', '20', '21', '22', '23', '24']
SENSORS_OFFICE = SENSORS_OFFICE1 + SENSORS_OFFICE2 + SENSORS_OFFICE3

SENSORS_MOBILE = ["02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25"]

INT_5s = '5sec/'
INT_10s = '10sec/'
INT_15s = '15sec/'
INT_30s = '30sec/'
INT_1min = '1min/'
INT_2min = '2min/'

Now, we can define the actual function to deal with fingerprints

In [ ]:
def get_val(sensor1, sensor2, feature, jv, flip='01'):
    if feature == 'audioFingerprint':
        if sensor2 == flip:
            return jv['results'][sensor1]['fingerprint_chunk1']
        else:
            return jv['results'][sensor1]['fingerprint_chunk2']
    if feature == 'noiseFingerprint':
        if type(jv['results'][sensor1]['fingerprint_noise_lev1']) is str:
            if sensor2 == flip:
                return jv['results'][sensor1]['fingerprint_noise_lev1']
            else:
                return jv['results'][sensor1]['fingerprint_noise_lev2']
        else:
            if sensor2 == flip:
                return ''.join([jv['results'][sensor1]['fingerprint_noise_lev1'][idx] for idx in sorted(jv['results'][sensor1]['fingerprint_noise_lev1'].keys())])
            else:
                return ''.join([jv['results'][sensor1]['fingerprint_noise_lev2'][idx] for idx in sorted(jv['results'][sensor1]['fingerprint_noise_lev2'].keys())])
    if feature == 'lux_miettinen':
        res = ''
        i = 128
        while i < len(jv["results"].keys()):
            x = sorted(jv["results"].keys())[i-1]
            res += jv["results"][x]
            i += 128
        i = min(i, len(jv["results"].keys()))
        x = sorted(jv["results"].keys())[i-1]
        res += jv["results"][x]
        return res

def import_fps(sensors, scenario, feature, interval):
    """Load results from a specific file and return them as python dict."""
    rv = {}
    if scenario == CAR_EXP:
        if feature is not 'lux_miettinen':
            pattern = BASE_DIR + scenario + 'Sensor-01/audio/' + feature + "/" + interval + 'Sensor-{}.json.gz'
            for sen in sensors:
                if sen == '01':
                    path = pattern.format('02')
                else:
                    path = pattern.format(sen)
                with gzip.open(path, 'rt') as fo:
                    j = json.loads(fo.read())
                    rv[sen] = []
                    for res in j['results']:
                        rv[sen].append(get_val(res, sen, feature, j))
        else:
            pattern = BASE_DIR + scenario + 'Sensor-{}/lux/lux_miettinen/' + interval + 'delta_abs-10.0/delta_rel-0.1/fp_len-128/result.json.gz'
            for sen in sensors:
                path = pattern.format(sen)
                with gzip.open(path, 'rt') as fo:
                    j = json.loads(fo.read())
                    rv[sen] = [get_val(None, sen, feature, j)]
    elif scenario == MOBILE_EXP:
        if feature is not 'lux_miettinen':
            pattern = BASE_DIR + scenario + 'Sensor-02/audio/' + feature + "/" + interval + 'Sensor-{}.json.gz'
            for sen in sensors:
                if sen == '02':
                    path = pattern.format('03')
                else:
                    path = pattern.format(sen)
                with gzip.open(path, 'rt') as fo:
                    j = json.loads(fo.read())
                    rv[sen] = []
                    for res in j['results']:
                        rv[sen].append(get_val(res, sen, feature, j, flip="02"))
        else:
            pattern = BASE_DIR + scenario + 'Sensor-{}/lux/lux_miettinen/' + interval + 'delta_abs-10.0/delta_rel-0.1/fp_len-128/result.json.gz'
            for sen in sensors:
                path = pattern.format(sen)
                with gzip.open(path, 'rt') as fo:
                    j = json.loads(fo.read())
                    rv[sen] = [get_val(None, sen, feature, j, flip="02")]
    if scenario == OFF_EXP:
        if feature is not 'lux_miettinen':
            for day in ['1_0-24h', '2_24-48h', '3_48-72h', '4_72-96h','5_96-120h', '6_120-144h', '7_144-168h']:
                pattern = BASE_DIR + scenario + 'audio/{}/Sensor-01/audio/' + feature + "/" + interval + 'Sensor-{}.json.gz'
                for sen in sensors:
                    if sen == '01':
                        path = pattern.format(day, '02')
                    else:
                        path = pattern.format(day, sen)
                    with gzip.open(path, 'rt') as fo:
                        j = json.loads(fo.read())
                        if sen not in rv:
                            rv[sen] = []
                        for res in j['results']:
                            rv[sen].append(get_val(res, sen, feature, j))
        else:
            pattern = BASE_DIR + scenario + 'Sensor-{}/lux/lux_miettinen/' + interval + 'delta_abs-10.0/delta_rel-0.1/fp_len-128/result.json.gz'
            for sen in sensors:
                path = pattern.format(sen)
                with gzip.open(path, 'rt') as fo:
                    j = json.loads(fo.read())
                    rv[sen] = [get_val(None, sen, feature, j)]
    return rv

Use the functions defined above to evaluate and plot the bit distributions for the schemes in question.

In [ ]:
for scenario, sensors in [(CAR_EXP, SENSORS_CAR), (OFF_EXP, SENSORS_OFFICE), (MOBILE_EXP, SENSORS_MOBILE)]:
    for interval in reversed([INT_5s, INT_10s, INT_15s, INT_30s, INT_1min, INT_2min]):
        a = import_fps(sensors, scenario, 'audioFingerprint', interval)
        for i in a.keys():
            keys = a[i]
            print(scenario, interval, i)
            plot_rand_walk(keys, bits=496, dist_xlim=[-200, 200], save_distribution_to='/home/seemoo/plots/img/%saudioFingerprint/audio/sensor-%s-%s-full.eps' % (scenario, i, interval[:-1]))

            keys = []
            for key in a[i]:
                for k in range(16):
                    tkey = key[k*31:k*31+31]
                    keys.append(tkey)
            plot_rand_walk(keys, bits=31, save_distribution_to='/home/seemoo/plots/img/%saudioFingerprint/audio/sensor-%s-%s-sub.eps' % (scenario, i, interval[:-1]))